<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Импортируем-необходимые-библиотеки." data-toc-modified-id="Импортируем-необходимые-библиотеки.-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Импортируем необходимые библиотеки.</a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

### Импортируем необходимые библиотеки.

In [1]:
# !pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

# warnings
# warnings.filterwarnings("ignore")

## Подготовка

Создадим экземпляры необходимых классов из библиотеки DistilBert.

In [3]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

In [4]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Загрузим данные, предусмотрим различные способы расположения файлов.

In [5]:
try:
    toxic_comments = pd.read_csv('toxic_comments.csv')
except:
    toxic_comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

Проверим читаемость данных.

In [6]:
toxic_comments.sample(3)

,text,toxic
110451,"""Uhm, Toya's name is spelled """"T-o-y-a"""" in th...",0
89525,Or at least review the timing of Moreschi's ob...,0
88498,"""Shouldn't this be moved somewhere? I know the...",0


In [7]:
toxic_comments.shape

(159571, 2)

Проверим соотношение классов.

In [8]:
toxic_comments['toxic'].mean()

0.10167887648758234

Данные не сбалансированы.

Из общего датасета отберем массив данных для обучения и тестирования модели.

In [9]:
toxic_sample = toxic_comments.sample(32000)

На этом подготовка данных завершена, приступим к обучению.

## Обучение

Выполним токенизацию текстов.

In [10]:
%%time

tokenized = toxic_sample['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)) # , max_lenght=512

CPU times: user 1min 10s, sys: 396 ms, total: 1min 11s
Wall time: 1min 14s


Найдем максимальную длину получившихся последовательностей.

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

Приведем все последовальности к одной размерности.

In [12]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [13]:
attention_mask = np.where(padded != 0, 1, 0)

In [14]:
attention_mask.shape

(32000, 512)

Найдем эмбеддинги.

In [15]:
%%time

batch_size = 80
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda() 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/400 [00:00<?, ?it/s]

CPU times: user 18min 5s, sys: 3.43 s, total: 18min 9s
Wall time: 18min 7s


In [16]:
len(embeddings)

400

Объединив эмбеддинги получим искомые признаки, на которых будем обучать логическую регрессию.

In [17]:
features = np.concatenate(embeddings)

In [18]:
features.shape

(32000, 768)

Разобьем данные для обучения и тестирования в соотношении 4:1.

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features, toxic_sample['toxic'], test_size=0.2)

In [20]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(25600, 768) (6400, 768) (25600,) (6400,)


Для получения более стабильных результатов применим кросс-валидацию.

In [21]:
lr = LogisticRegression(max_iter=1000) # , class_weight='balanced'

In [22]:
parametrs = {'C': np.arange(.5, 2, .5)}

In [23]:
%%time

lr_grid = GridSearchCV(lr, parametrs, cv=5, scoring='f1', n_jobs=-1)
lr_grid.fit(X_train, y_train)
best_lr = lr_grid.best_estimator_
print(f'best_params: {lr_grid.best_params_}, best_score: {lr_grid.best_score_}')

best_params: {'C': 1.5}, best_score: 0.727301290641033
CPU times: user 52.6 s, sys: 4.85 s, total: 57.5 s
Wall time: 4min 13s


Проверим на тестовых данных работу модели с лучшими параметрами.

In [24]:
y_pred_test = best_lr.predict(X_test)

In [25]:
f1_score(y_test, y_pred_test)

0.745003996802558

Результат работы модели по метрике F1 близок к желаемому. При округлении до 2х знаков соответствует требованим задания.

Проделаем аналогичные действия с масштабированными признаками.

In [26]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [27]:
lr_scaled = LogisticRegression(max_iter=1000) # , class_weight='balanced'

In [28]:
parametrs_scaled = {'C': np.arange(.01, .2, .05)} 

In [29]:
%%time

lr_grid_scaled = GridSearchCV(lr, parametrs_scaled, cv=5, scoring='f1', n_jobs=-1)
lr_grid_scaled.fit(X_train_scaled, y_train)
best_lr_scaled = lr_grid_scaled.best_estimator_
print(f'best_params: {lr_grid_scaled.best_params_}, best_score: {lr_grid_scaled.best_score_}')

best_params: {'C': 0.060000000000000005}, best_score: 0.7244033494880976
CPU times: user 11.3 s, sys: 1.08 s, total: 12.4 s
Wall time: 1min 27s


In [30]:
y_pred_test_scaled = best_lr_scaled.predict(X_test_scaled)

In [31]:
f1_score(y_test, y_pred_test_scaled)

0.7356687898089171

Результаты работы модели по метрике F1 немного ухудшились.

## Выводы

В результате работы получена модель, дающая приемлемый результат за разумное время.

Результат работы модели по метрике F1: 0.75.

Комбинация алгоритмов BERT и LogisticRegression позволяет получить неплохой результат для задач классификации текстов "из коробки", не прибегая к сложным приемам "ручной" предобработки признаков. Это достигается за счет использования ранее предобученных моделей.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны